In [ ]:
import pandas as pd
import ast
import numpy as np
import matplotlib
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plots
from mpl_toolkits.mplot3d import Axes3D
import math
import scipy.stats as stats
plots.style.use('fivethirtyeight')
from scipy.stats import f
import scipy
from scipy.stats import ttest_ind
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt
import matplotlib.pyplot as mp
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import matplotlib.pyplot as P
import seaborn as sns
import matplotlib as mpl
%reload_ext autoreload
%autoreload 2
import plotly.express as px
from wordcloud import WordCloud

In [ ]:
#Load data
df=pd.read_csv('',encoding = "Latin-1")

In [ ]:
df['incentivized'].replace({'NominalGift':'Incentive','VendorReferredIncentivized':'Incentive','NoIncentive':' NoIncentive','NonNominalGift':'NoIncentive','VendorReferred':'NoIncentive'}, inplace=True)
df[['incentivized']].head(10)

In [ ]:
df['incentivized'].value_counts()

In [ ]:
colors = ['darkblue', 'skyblue']

df['incentivized'].value_counts().plot(kind='barh', color=colors, fontsize=10, figsize=(6,2))
plt.ylabel("Incentivized", fontweight='bold', fontsize=10)
plt.xlabel("Review Count", fontweight='bold', fontsize=10)
plt.title("Review Count Based on Receiving Reward or Not", y=1.02, fontweight='bold', fontsize=10)
plt.tight_layout()
plt.show()

##**Modify the code to get the visualizations for "Ease of Use", "Features", "Customer Support" and "Likelihood to Recommend"**

In [ ]:
dfov = df.groupby(['overAllRating', 'incentivized'])['incentivized'].count()
dfov = dfov.unstack('incentivized', fill_value=0)

dfvf = df.groupby(['value_for_money', 'incentivized'])['incentivized'].count()
dfvf = dfvf.unstack('incentivized', fill_value=0)

colors = ['skyblue','darkblue']

fig, axs = plt.subplots(ncols=2, figsize=(13,6))
fig.subplots_adjust(wspace=0.4)

dfov.plot(kind='barh', ax=axs[0], color=colors)
dfvf.plot(kind='barh', ax=axs[1], color=colors)

# Add count labels to each bar
for i in axs:
    for j in i.containers:
        i.bar_label(j, label_type='edge', fontweight='bold', fontsize=10, labels=j.datavalues.astype(int))

axs[0].set_title('Over All Rating', fontsize=18, fontweight='bold')
axs[0].set_xlabel('Count', fontsize=14, fontweight='bold')
axs[0].set_ylabel('Rate', fontsize=14, fontweight='bold')
axs[0].tick_params(axis='y', labelsize=12)

axs[1].set_title('Value for Money', fontsize=18, fontweight='bold')
axs[1].set_xlabel('Count', fontsize=14, fontweight='bold')
axs[1].set_ylabel('Rate', fontsize=14, fontweight='bold')
axs[1].tick_params(axis='y', labelsize=12)

for ax in axs:
    ax.legend().remove()

handles, labels = axs[1].get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', ncol=4, bbox_to_anchor=(0.5, -0.15))

plt.subplots_adjust(wspace=0.3)

plt.show()

##**Reviews over the years**

In [ ]:
df13 = df.groupby(['year','ReviewDescription_Sentiment', 'incentivized'])['incentivized'].count().reset_index(name='count')

df13 = df13.pivot(index='year', columns=['incentivized', 'ReviewDescription_Sentiment'], values='count')
df13.columns = ['NoIncentive_NEGATIVE', 'NoIncentive_POSITIVE', 'Incentive_NEGATIVE', 'Incentive_POSITIVE']

fig, ax = plt.subplots(figsize=(4,4))

ax.plot(df13.index, df13['Incentive_POSITIVE'], color='darkblue', label='Incentive - Positive')
ax.plot(df13.index, df13['Incentive_NEGATIVE'], color='skyblue', label='Incentive - Negative')
ax.plot(df13.index, df13['NoIncentive_POSITIVE'], color='royalblue', label='No Incentive - Positive')
ax.plot(df13.index, df13['NoIncentive_NEGATIVE'], color='powderblue', label='No Incentive - Negative')

ax.set_xlabel('Year', fontweight='bold', fontsize=10)
ax.set_ylabel('Count of Review Description Sentiment', fontweight='bold', fontsize=10)
ax.set_title('Reviews over Years by Incentivized and Sentiment Status', fontweight='bold', fontsize=10)

ax.legend(loc='lower center', bbox_to_anchor=(0.5, -0.4), ncol=2, fontsize=12)
ax.tick_params(axis='x', labelsize=10)
ax.tick_params(axis='y', labelsize=10)


plt.show()

##**Reviews by time used (modify the code to get the results for "company size based on the number of employees")**

In [ ]:
df_filtered = df.loc[df['time_used'] != 'None']
df18 = df_filtered.groupby(['time_used', 'ReviewDescription_Sentiment', 'incentivized'])['incentivized'].count().reset_index(name='count')
df18 = df18.pivot(index='time_used', columns=['incentivized', 'ReviewDescription_Sentiment'], values='count')
df18.columns = ['NoIncentive_NEGATIVE', 'NoIncentive_POSITIVE', 'Incentive_NEGATIVE', 'Incentive_POSITIVE']
df18 = df18.sort_values('Incentive_POSITIVE')

fig, ax = plt.subplots(figsize=(10,4))

ax.plot(df18.index, df18['Incentive_POSITIVE'], color='darkblue', label='Incentive - Positive')
ax.plot(df18.index, df18['Incentive_NEGATIVE'], color='skyblue', label='Incentive - Negative')
ax.plot(df18.index, df18['NoIncentive_POSITIVE'], color='royalblue', label='No Incentive - Positive')
ax.plot(df18.index, df18['NoIncentive_NEGATIVE'], color='powderblue', label='No Incentive - Negative')

ax.set_xlabel('Time Used', fontweight='bold', fontsize=12)
ax.set_ylabel('Count of Review Description Sentiment', fontweight='bold', fontsize=12)
ax.set_title('Reviews by Time Used, Incentivized and Sentiment Status', fontweight='bold', fontsize=12)

ax.legend(loc='lower center', bbox_to_anchor=(0.5, -0.4), ncol=2, fontsize=12)
ax.tick_params(axis='x', labelsize=10)
ax.tick_params(axis='y', labelsize=10)

plt.show()

##**'Review Description', 'Pros' and 'Cons' sentiment count**

In [ ]:
dfs = df.groupby(['ReviewDescription_Sentiment', 'incentivized'])['incentivized'].count()
dfs = dfs.unstack('incentivized', fill_value=0)

dfp = df.groupby(['pros_Sentiment', 'incentivized'])['incentivized'].count()
dfp = dfp.unstack('incentivized', fill_value=0)

dfc = df.groupby(['cons_Sentiment', 'incentivized'])['incentivized'].count()
dfc = dfc.unstack('incentivized', fill_value=0)

colors = ['skyblue','darkblue' ]

# Create the horizontal bar plot
fig, axs = plt.subplots(ncols=3, figsize=(15,3))
fig.subplots_adjust(wspace=0.4)

dfs.plot(kind='barh', ax=axs[0], color=colors)
dfp.plot(kind='barh', ax=axs[1], color=colors)
dfc.plot(kind='barh', ax=axs[2], color=colors)

# Add count labels to each bar
for i in axs:
    for j in i.containers:
        i.bar_label(j, label_type='edge', fontweight='bold', fontsize=10, labels=j.datavalues.astype(int))

axs[0].set_title('Review Description', fontsize=18, fontweight='bold')
axs[0].set_xlabel('Count', fontsize=14, fontweight='bold')
axs[0].set_ylabel('Sentiment', fontsize=14, fontweight='bold')
axs[0].tick_params(axis='y', labelsize=12)

axs[1].set_title('Pros', fontsize=18, fontweight='bold')
axs[1].set_xlabel('Count', fontsize=14, fontweight='bold')
axs[1].set_ylabel('Sentiment', fontsize=14, fontweight='bold')
axs[1].tick_params(axis='y', labelsize=12)

axs[2].set_title('Cons', fontsize=18, fontweight='bold')
axs[2].set_xlabel('Count', fontsize=14, fontweight='bold')
axs[2].set_ylabel('Sentiment', fontsize=14, fontweight='bold')
axs[2].tick_params(axis='y', labelsize=12)


for ax in axs:
    ax.legend().remove()

handles, labels = axs[2].get_legend_handles_labels()

fig.legend(handles, labels, loc='lower center', ncol=4, bbox_to_anchor=(0.5, -0.35))

plt.subplots_adjust(wspace=0.7)

plt.show()

##**Top 20 words**

In [ ]:
from collections import Counter


# group the reviews
grouped_reviews = df.groupby(['ReviewDescription_Sentiment', 'incentivized'])

# create a word cloud
for name, group in grouped_reviews:
    sentiment, incentivized = name
    text = " ".join(review for review in group.preprocessed_ReviewDescription)
    wordcloud = WordCloud(background_color='white')
    wordcloud.generate(text)

    # Get a dictionary
    word_frequencies = wordcloud.process_text(text)

    # Sort the dictionary
    sorted_word_frequencies = dict(sorted(word_frequencies.items(), key=lambda item: item[1], reverse=True))

    # Select the top 20 words
    top_20_words = dict(Counter(sorted_word_frequencies).most_common(20))

    fig, ax = plt.subplots(1, 2, figsize=(12,6))
    ax[0].imshow(wordcloud, interpolation='bilinear')
    ax[0].axis("off")
    ax[0].set_title(f"{sentiment} Sentiment - Incentivized: {incentivized}",fontweight='bold', fontsize=12)

    ax[1].barh(list(top_20_words.keys()), list(top_20_words.values()), color='royalblue')
    ax[1].tick_params(axis='y', labelsize=12)

    ax[1].set_xlabel(f'{sentiment} {incentivized} Review Count', fontweight='bold', fontsize=12)
    ax[1].set_title('Top 20 Words by Incentivized and Sentiment Status', fontweight='bold', fontsize=12)

    plt.show()

In [ ]:
df_select=df[['incentivized','overAllRating', 'value_for_money', 'ease_of_use','features','customer_support','likelihood_to_recommend', 'frequency_of_use','time_used','source']]

plt.figure(figsize=(8,6))
sns.set_context('paper', font_scale=1.4)
df_select_mx=df_select.corr()
df_select_mx


df_select_mx.values[np.triu_indices_from(df_select_mx)] = 1
mask = np.zeros_like(df_select_mx)
mask[np.triu_indices_from(mask)] = True

sns.heatmap(df_select_mx, mask=mask, annot=True, cmap='Greens')

In [ ]:
df_select_mx = df_select_mx.where(np.triu(np.ones(df_select_mx.shape)).astype(np.bool))

# Set the diagonal to 1
np.fill_diagonal(df_select_mx.values, 1)

# Create the heatmap
fig, ax = plt.subplots(figsize=(4.5, 4.5))
ax=sns.heatmap(df_select_mx, annot=True, cmap='Blues', vmin=0, vmax=1, square=True, annot_kws={"size": 10})


plt.setp(ax.get_xticklabels(), fontsize=10)
#plt.setp(ax.get_xticklabels(), rotation=90, fontsize=10)
plt.setp(ax.get_yticklabels(),  fontsize=10)
plt.title('Correlation Among Review Features',fontweight='bold', fontsize=10)
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=10)
plt.show()

##**Top 20 trigrams**

In [ ]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import random

nltk.download('stopwords')
nltk.download('wordnet')

# Load dataset
df = pd.read_csv('')

# Select 5000 random samples
incentive_df = df[df['Incentivized'] == 'Incentive'].sample(n=5000, random_state=1)
noincentive_df = df[df['Incentivized'] == 'NoIncentive'].sample(n=5000, random_state=1)

# Preprocessing function
def preprocess(text):
    if not isinstance(text, str):
        return ''
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [word for word in words if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)


incentive_df['processed_review'] = incentive_df['preprocessed_ReviewDescription'].apply(preprocess)
noincentive_df['processed_review'] = noincentive_df['preprocessed_ReviewDescription'].apply(preprocess)

# Extract top n trigrams
def get_top_n_trigrams(corpus, n=40):
    vectorizer = TfidfVectorizer(ngram_range=(3, 3))
    X = vectorizer.fit_transform(corpus)
    scores = np.array(X.sum(axis=0)).ravel()
    indices = np.argsort(scores)[::-1]
    feature_names = np.array(vectorizer.get_feature_names_out())
    top_features = [(feature_names[i], scores[i]) for i in indices[:n]]
    return dict(top_features)

top_incentive_trigrams = get_top_n_trigrams(incentive_df['processed_review'])
top_noincentive_trigrams = get_top_n_trigrams(noincentive_df['processed_review'])

def plot_top_trigrams(top_trigrams, title):
    trigrams, freqs = zip(*top_trigrams.items())
    plt.figure(figsize=(10, 10.5))

    bars = plt.barh(trigrams, freqs, color='blue')

    plt.barh(trigrams, freqs)
    plt.xlabel('TF-IDF Score')
    plt.title(f'Top 40 Trigrams in {title} Review Description')

    for
        width = bar.get_width()
        label_x_pos = width + 0.01

        plt.text(label_x_pos, bar.get_y() + bar.get_height() / 2, f'{width:.2f}',
                 va='center')

    plt.gca().invert_yaxis()
    plt.show()

plot_top_trigrams(top_incentive_trigrams, 'Incentive')
plot_top_trigrams(top_noincentive_trigrams, 'NoIncentive')


##**Group Listing IDs**

In [ ]:
df['Incentivized'] = df['Incentivized'].str.upper()
df['ReviewDescription_Sentiment'] = df['ReviewDescription_Sentiment'].str.upper()
df['pros_Sentiment'] = df['pros_Sentiment'].str.upper()
df['cons_Sentiment'] = df['cons_Sentiment'].str.upper()

# Group by 'listing_id'
review_sentiment = df.groupby(['listing_id', 'Incentivized'])['ReviewDescription_Sentiment'].value_counts().unstack(fill_value=0)
pros_sentiment = df.groupby(['listing_id', 'Incentivized'])['pros_Sentiment'].value_counts().unstack(fill_value=0)
cons_sentiment = df.groupby(['listing_id', 'Incentivized'])['cons_Sentiment'].value_counts().unstack(fill_value=0)

category = review_sentiment.groupby('listing_id').apply(lambda x: 'Both (Incentive & NoIncentive)' if len(x) > 1 else ('Just Incentive' if 'INCENTIVE' in x.index.get_level_values(1) else 'Just NoIncentive'))
category.name = 'Category'

review_sentiment = review_sentiment.merge(category, on='listing_id')
pros_sentiment = pros_sentiment.merge(category, on='listing_id')
cons_sentiment = cons_sentiment.merge(category, on='listing_id')

review_count = review_sentiment.groupby('Category')['POSITIVE', 'NEGATIVE'].sum()
pros_count = pros_sentiment.groupby('Category')['POSITIVE', 'NEGATIVE'].sum()
cons_count = cons_sentiment.groupby('Category')['POSITIVE', 'NEGATIVE'].sum()
listing_count = review_sentiment.reset_index().groupby('Category')['listing_id'].nunique()

# Combine the results
result = pd.concat([listing_count, review_count, pros_count, cons_count], axis=1)
result.columns = ['Number of Listings', 'Review Description POSITIVE', 'Review Description NEGATIVE', 'Pros POSITIVE', 'Pros NEGATIVE', 'Cons POSITIVE', 'Cons NEGATIVE']

print(result)


In [ ]:
# Prepare columns for analysis
df['Incentivized'] = df['Incentivized'].str.upper()
df['ReviewDescription_Sentiment'] = df['ReviewDescription_Sentiment'].str.upper()
df['pros_Sentiment'] = df['pros_Sentiment'].str.upper()
df['cons_Sentiment'] = df['cons_Sentiment'].str.upper()

# Function to categorize listings (replace the dots with results from previous code)
def categorize_listing(row):
    if row['Incentivized_count'] == 2:
        return 'Both (... ID)'
    elif 'INCENTIVE' in row['Incentivized_types']:
        return 'Incentive (... ID)'
    else:
        return 'NoIncentive (... ID)'

listing_categories = df.groupby('listing_id')['Incentivized'].agg(['nunique', 'unique']).rename(columns={'nunique': 'Incentivized_count', 'unique': 'Incentivized_types'})
df = df.join(listing_categories, on='listing_id')
df['Category'] = df.apply(categorize_listing, axis=1)

# Group and count the sentiments
review_sentiment = df.groupby(['Category', 'ReviewDescription_Sentiment'])['listing_id'].count().unstack(fill_value=0)
pros_sentiment = df.groupby(['Category', 'pros_Sentiment'])['listing_id'].count().unstack(fill_value=0)
cons_sentiment = df.groupby(['Category', 'cons_Sentiment'])['listing_id'].count().unstack(fill_value=0)

colors = ['skyblue', 'darkblue']

fig, axs = plt.subplots(ncols=3, figsize=(15, 5), sharey=True)
fig.subplots_adjust(wspace=0.5)

def plot_bars(ax, data, title):
    bars = data.plot(kind='barh', ax=ax, color=colors, width=0.8)
    ax.set_title(title, fontweight='bold')
    ax.set_xlabel('Count', fontsize=14, fontweight='bold')

    max_bar_value = data.values.max()

    for bar in bars.patches:
        bar_value = int(bar.get_width())
        if bar_value > max_bar_value * 0.1:
            text_x_position = bar_value - (bar_value * 0.01)
            text_color = 'white'
            ha = 'right'
        else:
            text_x_position = bar_value + (max_bar_value * 0.01)
            text_color = 'black'
            ha = 'left'

        ax.text(text_x_position, bar.get_y() + bar.get_height() / 2,
                f'{bar_value}',
                va='center', ha=ha, color=text_color, fontweight='bold')
    ax.set_xlim(0, max_bar_value + (max_bar_value * 0.1))

plot_bars(axs[0], review_sentiment, 'Review Description Sentiment')
plot_bars(axs[1], pros_sentiment, 'Pros Sentiment')
plot_bars(axs[2], cons_sentiment, 'Cons Sentiment')

for ax in axs:
    plt.sca(ax)
    plt.yticks(fontweight='bold')
    ax.set_ylabel('Listing ID Category', fontweight='bold')

handles, labels = axs[2].get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.1))

plt.tight_layout()

plt.show()
